In [1]:
import pandas as pd

In [2]:
# pipeline data used in Draft Blueprint modeling
pipeline_dbp_version = pd.read_csv(
    r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\parcels_buildings_agents\development_pipeline_NP_2024-03-08.csv'
)
print('row count: {}'.format(len(pipeline_dbp_version)))
print('\nunique development_projects_id: {}\n'.format(pipeline_dbp_version['development_projects_id'].nunique()))
print(pipeline_dbp_version['action'].value_counts(dropna=False), '\n')
print(pipeline_dbp_version[['non_residential_sqft', 'residential_units']].sum(), '\n')
print(pipeline_dbp_version.groupby('action')[['non_residential_sqft', 'residential_units']].sum())

row count: 58723

unique development_projects_id: 58723

action
build    53713
add       5010
Name: count, dtype: int64 

non_residential_sqft    392460056.0
residential_units          439465.0
dtype: float64 

        non_residential_sqft  residential_units
action                                         
add               82823884.0            58031.0
build            309636172.0           381434.0


C:\Users\ywang\AppData\Local\Temp\ipykernel_4052\1866414638.py:2: DtypeWarning: Columns (3,4,6,7,8,9,14,22,23,26,27,34) have mixed types. Specify dtype option on import or set low_memory=False.
  pipeline_dbp_version = pd.read_csv(


In [3]:
# a function to modify the project list to ensure the earliest project on a parcel has "build" action

def ensure_earliest_proj_build(df):
    df = df.sort_values(["PARCEL_ID", "year_built"])
    prev_parcel_id = None
    for index, rec in df.iterrows():
        if rec.PARCEL_ID != prev_parcel_id:
            df.loc[index, "action"] = "build"
        prev_parcel_id = rec.PARCEL_ID
    
    return df

In [4]:
# apply the modification
pipeline_updated = ensure_earliest_proj_build(pipeline_dbp_version)

print(pipeline_updated['action'].value_counts(dropna=False), '\n')
print(pipeline_updated[['non_residential_sqft', 'residential_units']].sum(), '\n')
print(pipeline_updated.groupby('action')[['non_residential_sqft', 'residential_units']].sum())

action
build    54503
add       4220
Name: count, dtype: int64 

non_residential_sqft    392460056.0
residential_units          439465.0
dtype: float64 

        non_residential_sqft  residential_units
action                                         
add               56993041.0            38754.0
build            335467015.0           400711.0


In [ ]:
# drop "geom_id" column
pipeline_updated.drop(['geom_id'], axis=1, inplace=False)
print(list(pipeline_updated))

In [5]:
# drop "geom_id" column
pipeline_updated.drop(['geom_id'], axis=1, inplace=False)
print(list(pipeline_updated))

['development_projects_id', 'OBJECTID', 'raw_id', 'building_name', 'site_name', 'action', 'address', 'city', 'zip', 'county', 'x', 'y', 'geom_id', 'year_built', 'building_type_det', 'building_type', 'building_type_id', 'development_type_id', 'building_sqft', 'non_residential_sqft', 'residential_units', 'unit_ave_sqft', 'tenure', 'rent_type', 'stories', 'parking_spaces', 'average_weighted_rent', 'last_sale_year', 'last_sale_price', 'deed_restricted_units', 'source', 'PARCEL_ID', 'ZONE_ID', 'edit_date', 'editor', 'data_source', 'parcel_id_basis', 'building_id_basis', 'residential_sqft', 'preserved_units', 'inclusionary_units', 'subsidized_units']


In [6]:
# write out
pipeline_updated.to_csv(
    r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\parcels_buildings_agents\development_pipeline_2024-07-31.csv', index=False
)